# Access to IonQ, Rigetti and OQC via AWS Braket

Amazon Braket is a fully managed quantum computing service that offers access to different types of quantum computing hardware and quantum circuit simulators.

AWS Braket serves as a portal to a variety of quantum hardware, currently including:
 - Quantum annealers (from D-WAVE);
 - Trapped-ion gate-model devices (from IonQ);
 - Superconducting gate-model devices (from Rigetti and OQC)
 - Programmable photonic processor (from XANADU)
 
It also offers access to two powerful simulators: a state-vector simulator with a capacity of 34 qubits, and a tensor-network simulator with a capacity of 50 qubits for suitably sparse circuits.


## Get Access to AWS Braket

Amazon Braket is part of the AWS Free Tier. The AWS Free Tier gives you one free hour of quantum circuit simulation time per month during the first twelve months of use. This applies to simulation time on SV1, DM1, TN1, or any combination of those three managed quantum circuit simulators, or use of simulators through the Hybrid Jobs feature. Standard rates for incremental usage apply after the free hour has been used. Use of other Amazon Braket resources such as managed notebooks, hybrid job instances, and quantum computers will incur costs.

To explore Free Tier products with a new AWS account click [here](https://aws.amazon.com/braket/pricing/?loc=ft#AWS_Free_Tier). Plese read the infromation carefully.

To sign up for a free account click [here](https://portal.aws.amazon.com/billing/signup?redirect_url=https%3A%2F%2Faws.amazon.com%2Fregistration-confirmation#/start/email). Note: You will need a credit card to sign up. Amazon states specifically:"We will not charge you for usage below AWS Free Tier limits. We may temporarily hold up to $1 USD (or an equivalent amount in local currency) as a pending transaction for 3-5 days to verify your identity." The sign up process is 5 steps long. At the end choose "Basic support - Free" as your plan. Once your sign-up process is completed you will receive confirmation emails. You can now access the "AWS Management Console" by signing in as a "Root User". Now, for the next 12 months, you'll have free access to core AWS compute, storage, database, and application services within the limits of the [Free Tier](https://aws.amazon.com/braket/pricing/?loc=ft#AWS_Free_Tier).

Once you signed into your "AWS Management Console" (for me it is https://us-west-1.console.aws.amazon.com/) you can access Amazon Braket's Quantum Technologies by clicking on the orange "Services" tab in the top left corner. Scroll down to select "Quantum Technologies" and click on "Amazon Braket". Now under "Devices" you will see all quantum computers Amazon Braket gives you access to.

#### Note: We will only need to compile our circuits to a given backend, not run it on the actual computer. So no charges will be needed to be made towards your account!


### AWS Access Key ID

Access keys are needed to make programmatic calls to AWS from the AWS CLI, Tools for PowerShell, AWS SDKs, or direct AWS API calls. 

To find/generate your access key (access key ID and secret access key), click on your "name" next to "Global" tab on top. Here you can see your account ID. The key ID and secret access key are under "Security credentials". Click on "Security credentials" and then click on "Access keys (access key ID and secret access key). 

If you don't have one yet, generate one. Else they will be listed in a table. 

[From](https://docs.aws.amazon.com/braket/latest/developerguide/braket-devices.html), the device names we can access look like this:

In [5]:
# read aws tokens
with open('tokens/rootkey.csv') as file:
    lines = file.readlines()
    aws_access_key_id = lines[0].split('=')[1].strip()
    aws_secret_access_key = lines[1].split('=')[1].strip()

In [1]:
#device = AwsDevice('arn:aws:braket:::device/quantum-simulator/amazon/sv1')              #SV1
# device = LocalSimulator()                                                             #Local State Vector Simulator
# device = LocalSimulator("default")                                                    #Local State Vector Simulator
# device = LocalSimulator(backend="default")                                            #Local State Vector Simulator
# device = LocalSimulator(backend="braket_sv")                                          #Local State Vector Simulator
# device = LocalSimulator(backend="braket_dm")                                          #Local Density Matrix Simulator
# device = AwsDevice('arn:aws:braket:::device/quantum-simulator/amazon/tn1')            #TN1
# device = AwsDevice('arn:aws:braket:::device/quantum-simulator/amazon/dm1')            #DM1
# device = AwsDevice('arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6')                   #D-Wave 2000Q
# device = AwsDevice('arn:aws:braket:::device/qpu/d-wave/Advantage_system4')            #D-Wave Advantage_system4.1
# device = AwsDevice('arn:aws:braket:us-west-2::device/qpu/d-wave/Advantage_system6')   #D-Wave Advantage_system6.1
# device = AwsDevice('arn:aws:braket:::device/qpu/ionq/ionQdevice')                     #IonQ
# device = AwsDevice('arn:aws:braket:::device/qpu/rigetti/Aspen-11')                    #Rigetti Aspen-11
# device = AwsDevice('arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-1')          #Rigetti Aspen M-1
# device = AwsDevice('arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy')                   #OQC Lucy
# device = AwsDevice('arn:aws:braket:us-east-1::device/qpu/xanadu/Borealis')            #Xanadu Borealis



Now, we need to create AWS braket clients for the following regions to get access to gate-based IonQ, Rigetti, and OQC devices.

In [7]:
### Creat the AWS braket client

import boto3
# Create the braket clients

#for us east
boto_session_east = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name='us-east-1'
)

#for eu west
boto_session_eu_west = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name='eu-west-2'
)

#for us west
boto_session_west = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name='us-west-1'
)

## Amazon Braket integration example for pytket

Using the `pytket-braket` extension, TKET can use Braket to run circuits on all its gate-model devices and simulators. TKET does not support annealers and photonic processors.

Because Braket devices provide detailed calibration data, we can take full advantage of TKET’s noise-aware placement and routing. A TKET device constructed via a `BraketBackend` contains the full error characteristics of the device.

### Getting started

First, install the latest versions of pytket and pytket-braket:

In [8]:
# pip install pytket

In [9]:
# pip install pytket-braket

In [10]:
from pytket.extensions.braket import BraketBackend

In [11]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

circ = Circuit(2)
circ.H(0)
circ.CX(0,1)

render_circuit_jupyter(circ)

# Access to IonQ

IonQ’s trapped ion QPUs are built on a chain of trapped 171Yb+ ions, spatially confined via a microfabricated surface electrode trap within a vacuum chamber. Gates are performed via a two-photon Raman transition using a pair of counter-propagating beams from a mode-locked pulsed laser. This allows for high-quality single and two-qubit transitions and all-to-all connectivity. Initialization is performed via optical pumping, and readout is performed with a combination of a resonant laser, a high numeric aperture lens, and photomultiplier tubes.

The device_type, provider and device arguments all come from the device’s ARN as shown on the Amazon Braket page describing the device. In this case, the ARN is 'arn:aws:braket:::device/qpu/ionq/ionQdevice' and the Region: us-east-1

The device page also shows whether the device is currently available or when it will next be. Now we need to compile the circuit for the backend device. At a minimum, this involves converting it to the target gate set and placing the qubits. We can control the optimisation level used in compilation, but let’s just use the default, level = 0.

In [12]:
from braket.aws import AwsSession
aws_session = AwsSession(boto_session=boto_session_east)

In [13]:
ionq_backend = BraketBackend(
device_type='qpu', 
provider='ionq',
device='ionQdevice',
aws_session=aws_session
)

In [14]:
ionq_compiled_circ = ionq_backend.get_compiled_circuit(circ,0)   #change 0 to 1 or 2 for additional optimization
render_circuit_jupyter(ionq_compiled_circ)

In [15]:
from pytket.circuit import OpType

print('Circuit depth:', ionq_compiled_circ.depth())
print('CX depth:     ', ionq_compiled_circ.depth_by_type(OpType.CX))

Circuit depth: 2
CX depth:      1


# Access to OQC - Oxford Quantum Circuits — Lucy

Oxford Quantum Circuits (OQC) quantum computers are universal, gate-based machines based on superconducting qubits built using proprietary 'Coaxmon' technology. The Coaxmon design has a scalable, three-dimensional architecture that introduces the qubit control electronics perpendicular to the plane of the qubits. This simplification in the architecture makes it easier to scale to higher qubit numbers than conventional 2D circuits. Lucy, OQC's first quantum computer available on Braket, offers an 8 qubit QPU with a topology of a ring where each qubit is connected to its 2 nearest neighbors.

Device ARN" ' arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy'

Region: eu-west-2

In [16]:
aws_session2 = AwsSession(boto_session=boto_session_eu_west)

In [17]:
oqc_backend = BraketBackend(
device_type='qpu',
provider='oqc',
device='Lucy',
aws_session=aws_session2
)

ValueError: QPU 'arn:aws:braket:::device/qpu/oqc/Lucy' not found

### NOTE: In case Lucy is not available online, you will get the following error message: "ValueError: QPU 'arn:aws:braket:::device/qpu/oqc/Lucy' not found"

# Access to Rigetti — Aspen-11

Rigetti quantum processors are universal, gate-model machines based on tunable superconducting qubits.

Rigetti's latest Aspen chips are based on scalable 40-qubit node and feature enhanced readout capabilities that contribute to better overall circuit fidelities independent of depth and width. Further system improvements and device characteristics include a speedup in quantum processing times, fast gate times for multiple entangling gate families, rapid sampling via active register reset, and parametric control.

Device ARN" 'arn:aws:braket:::device/qpu/rigetti/Aspen-11'

Region: us-west-1

In [18]:
aws_session3 = AwsSession(boto_session=boto_session_west)

In [19]:
aspen11_backend = BraketBackend(
device_type='qpu',
provider='rigetti',
device='Aspen-11',
aws_session=aws_session3
)

In [20]:
aspen11_compiled_circ = aspen11_backend.get_compiled_circuit(circ,0)
render_circuit_jupyter(aspen11_compiled_circ)

In [21]:
from pytket.circuit import OpType

print('Circuit depth:', aspen11_compiled_circ.depth())
print('CX depth:     ', aspen11_compiled_circ.depth_by_type(OpType.CX))

Circuit depth: 2
CX depth:      1


# Access to Rigetti -M-1

Rigetti's latest Aspen-M processor is based on scalable multi-chip technology, and features enhanced readout capabilities that contribute to better overall circuit fidelities independent of depth and width. Further system improvements and device characteristics include a speedup in quantum processing times, fast gate times for multiple entangling gate families, rapid sampling via active register reset, and parametric control.

Device ARN" 'arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-1'

Region: us-west-1

In [22]:
aspen11_backend = BraketBackend(
device_type='qpu',
provider='rigetti',
device='Aspen-M-1',
aws_session=aws_session3
)

ValueError: QPU 'arn:aws:braket:::device/qpu/rigetti/Aspen-M-1' not found

### NOTE: In case Lucy is not available online, you will get the following error message: "ValueError: QPU 'arn:aws:braket:::device/qpu/rigetti/Aspen-M-1' not found